# 05 - Delta Hedging Error Lab

Study discrete hedging error versus rebalance frequency and transaction costs under GBM.

In [5]:
%cd /content
!rm -rf interactive_portfolio_optimization
!git clone https://github.com/basarr/interactive_portfolio_optimization.git
!ls /content/interactive_portfolio_optimization


/content
Cloning into 'interactive_portfolio_optimization'...
remote: Enumerating objects: 79, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 79 (delta 30), reused 42 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (79/79), 50.45 KiB | 1.33 MiB/s, done.
Resolving deltas: 100% (30/30), done.
data  notebooks  pyproject.toml  README.md  results  src  tests


In [6]:
from pathlib import Path
print((Path("/content/interactive_portfolio_optimization/src")).exists())
print((Path("/content/interactive_portfolio_optimization/pyproject.toml")).exists())


True
True


In [7]:
from pathlib import Path
import sys

ROOT = Path("/content/interactive_portfolio_optimization")
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))
print("ROOT OK:", ROOT)


ROOT OK: /content/interactive_portfolio_optimization


In [8]:
from pathlib import Path
import sys

ROOT = Path.cwd()
if not (ROOT / 'src').exists():
    ROOT = ROOT.parent
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

import pandas as pd

from src.config import config_dict
from src.hedging import hedging_experiment_grid, simulate_delta_hedge_gbm
from src.plotting import plot_hedging_error_hist, plot_hedging_tradeoff

cfg = config_dict(fast_mode=True)


In [10]:
from pathlib import Path
import sys

ROOT = Path("/content/interactive_portfolio_optimization")
if not (ROOT / "src").exists():
    raise FileNotFoundError(f"Bad ROOT: {ROOT}")

for p in [ROOT/"results", ROOT/"results"/"tables", ROOT/"results"/"figures", ROOT/"results"/"logs", ROOT/"results"/"reports"]:
    p.mkdir(parents=True, exist_ok=True)

if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

print("ROOT:", ROOT)
print("tables exists:", (ROOT/"results"/"tables").exists())


ROOT: /content/interactive_portfolio_optimization
tables exists: True


In [11]:
grid = hedging_experiment_grid(
    S0=cfg['S0'], K=cfg['K'], r=cfg['R'], q=cfg['Q'], sigma=cfg['SIGMA'], T=cfg['T'],
    n_paths=8_000, n_steps=252,
    rebalance_list=cfg['HEDGE_REBALANCE_STEPS'],
    tx_cost_list=[0.0, 0.001, 0.003],
    option_type='call',
    seed=cfg['SEED'],
)
grid.to_csv(ROOT / 'results' / 'tables' / 'hedging_error_summary.csv', index=False)
plot_hedging_tradeoff(grid, ROOT / 'results' / 'figures' / 'hedging_tradeoff.png')
grid.head()


,rebalance_every_k_steps,tx_cost_per_dollar,mean_error,std_error,q05,q95
0,1,0.000,0.001985,0.439133,-0.718916,0.715099
1,1,0.001,-0.622575,0.489120,-1.511537,0.052750
2,1,0.003,-1.871695,0.744647,-3.237290,-0.835132
3,2,0.000,0.002015,0.611143,-1.008621,1.009791
4,2,0.001,-0.473483,0.635437,-1.585132,0.484121


In [12]:
base = simulate_delta_hedge_gbm(
    S0=cfg['S0'], K=cfg['K'], r=cfg['R'], q=cfg['Q'], sigma=cfg['SIGMA'], T=cfg['T'],
    n_paths=8_000, n_steps=252, rebalance_every_k_steps=5,
    option_type='call', tx_cost_per_dollar=0.0, seed=cfg['SEED']
)
plot_hedging_error_hist(base['errors'], ROOT / 'results' / 'figures' / 'hedging_error_hist.png')
print('Mean error:', base['mean_error'])
print('Std error:', base['std_error'])


Mean error: -0.00503362965521401
Std error: 0.9539140957106874


# Notebook 05 Summary

We analyzed discrete-time delta hedging for a short European call under geometric Brownian motion (GBM), focusing on the impact of rebalancing frequency and transaction costs on hedging error.

Model and hedge:

The underlying asset follows:

$dS_t = (r - q) S_t dt + \sigma,S_t  dW_t $

At each rebalance time, the hedge ratio is given by the Black–Scholes delta:
$\Delta_t = \frac{\partial V}{\partial S}(S_t,t).$

Rebalancing incurs proportional transaction costs:


$\text{Cost}t = c|\Delta_t - \Delta_{t^-}|S_t$

and hedging error is measured as the final hedged portfolio P&L after option settlement.

What we checked:

1.	Discrete delta hedging accuracy as rebalancing frequency increases.

2.	The effect of transaction costs on mean hedging P&L and tail risk.

3.	The trade-off between discretization error and cost drag.


Findings:

•	With very frequent rebalancing ($k=1$) and zero transaction costs, the mean hedging error is close to zero (approximately $0.002$) and dispersion is lowest (standard deviation around $0.44$), indicating close approximation to continuous-time replication.

•	Introducing transaction costs while keeping frequent rebalancing leads to strongly negative average P&L (approximately $-0.62$ for $c=0.001$ and $-1.87$ for $c=0.003$) and substantially worse downside outcomes.

•	In the base configuration ($k=5$, $c=0$), the mean error remains near zero (approximately $-0.005$), but variability is still material (standard deviation around $0.95$), reflecting residual discretization and gamma risk.

Key takeaways:
•	Discrete delta hedging converges toward the continuous-time Black–Scholes benchmark as rebalancing becomes more frequent.
•	Transaction costs introduce a systematic negative bias when hedging too often.
•	Less frequent rebalancing reduces costs but leaves larger unhedged price movements.
•	Effective hedging reflects a trade-off between discretization risk and trading costs, rather than perfect replication.